In [ ]:
import numpy as np
import pathlib 

import matplotlib.pyplot as plt

import torch

from torch import optim

import sys
sys.path.insert(0, '../')
import sdss_dataset_lib
import sdss_psf

from astropy.io import fits
from astropy.wcs import WCS

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

import os

from psf_transform_lib import get_psf_loss
import psf_transform_lib2
import fitsio

In [ ]:
# load data
bands = [2, 3]
x0 = 630
x1 = 310
sdss_hubble_data = sdss_dataset_lib.SDSSHubbleData(x0 = x0,
                                                    x1 = x1, 
                                                    bands = bands)


In [ ]:
# the full image
plt.matshow(sdss_hubble_data.sdss_image_full[0])

In [ ]:
# check the hubble coordinates overlap with the globular cluster
plt.matshow(sdss_hubble_data.sdss_image_full[0])
plt.plot(sdss_hubble_data.locs_full_x1, 
         sdss_hubble_data.locs_full_x0, alpha = 0.2)

In [ ]:
# check patch 

for i in range(len(bands)):
    plt.matshow(sdss_hubble_data.sdss_image[i])
    plt.colorbar()


In [ ]:
# check alignment between bands 

if len(bands) > 1: 
    band_diff = (sdss_hubble_data.sdss_image[1]) - \
                (sdss_hubble_data.sdss_image[0])

    plt.matshow(band_diff, vmax = band_diff.abs().max(), vmin = -band_diff.abs().max(), 
                cmap = plt.get_cmap('bwr'))

    plt.colorbar()

# SDSS PSF

In [ ]:
psf_dir = '../data/'
psf_r = fitsio.FITS(psf_dir + 'sdss-002583-2-0136-psf-r.fits')[0].read()
psf_i = fitsio.FITS(psf_dir + 'sdss-002583-2-0136-psf-i.fits')[0].read()

# Power law PSF

In [ ]:
psfield_file = '../../celeste_net/sdss_stage_dir/2583/2/136/psField-002583-2-0136.fit'

In [ ]:
init_psf_params = torch.zeros(len(bands), 6)
for i in range(len(bands)): 
    init_psf_params[i] = psf_transform_lib2.get_psf_params(
                                    psfield_file, 
                                    band = bands[i])

In [ ]:
power_law_psf = psf_transform_lib2.PowerLawPSF(init_psf_params)

In [ ]:
init_psf = power_law_psf.forward().detach()

In [ ]:
plt.matshow(init_psf[0, 46:55, 46:55])
plt.colorbar()

# get background

In [ ]:
background_bias = psf_transform_lib2.BackgroundBias(torch.zeros(len(bands), 3))

# Get my simulator

In [ ]:
import simulated_datasets_lib

In [ ]:
sky_intensity = sdss_hubble_data.sdss_background.reshape(len(bands), -1).mean(1)

simulator = simulated_datasets_lib.StarSimulator(psf = init_psf,
                                    slen = sdss_hubble_data.slen, 
                                    transpose_psf = False, 
                                    background = sdss_hubble_data.sdss_background)

# Check reconstruction with all of the stars

In [ ]:
_fluxes = sdss_hubble_data.fluxes.unsqueeze(0)
_locs = sdss_hubble_data.locs.unsqueeze(0)
_n_stars = torch.Tensor([len(sdss_hubble_data.locs)]).type(torch.LongTensor)
                                                               
recon_mean_dense = simulator.draw_image_from_params(locs = _locs, 
                                                fluxes = _fluxes, 
                                                n_stars = _n_stars, 
                                                add_noise = False) 

In [ ]:
for i in range(len(bands)): 
    f, axarr = plt.subplots(1, 3, figsize=(16, 4))

    observed = sdss_hubble_data.sdss_image
    im0 = axarr[0].matshow(observed[i])
    f.colorbar(im0, ax=axarr[0])
    axarr[0].set_title('observed, band = ' + str(bands[i]))


    im1 = axarr[1].matshow(recon_mean_dense[0, i])
    f.colorbar(im1, ax=axarr[1])
    axarr[1].set_title('recon, band = ' + str(bands[i]))

    residual = recon_mean_dense[0,i] - observed[i]
    foo = (residual / observed[i])
    im2 = axarr[2].matshow(foo, vmax = foo.abs().max(), vmin = - foo.abs().max(), cmap = plt.get_cmap('bwr'))
    f.colorbar(im2, ax=axarr[2])
    axarr[2].set_title('recon - obse, band = ' + str(bands[i]))

# Reconstruction with only bright stars

In [ ]:
fmin = 1000.

In [ ]:
which_bright = sdss_hubble_data.fluxes[:, 0] > fmin

_locs = sdss_hubble_data.locs[which_bright].unsqueeze(0)
_fluxes = sdss_hubble_data.fluxes[which_bright].unsqueeze(0)
_n_stars = torch.Tensor([len(_locs[0])]).type(torch.LongTensor)

recon_mean = simulator.draw_image_from_params(locs = _locs, 
                                                fluxes = _fluxes, 
                                                n_stars = _n_stars, 
                                                add_noise = False) 

In [ ]:
for i in range(len(bands)): 
    f, axarr = plt.subplots(1, 3, figsize=(16, 4))

    observed = sdss_hubble_data.sdss_image
    im0 = axarr[0].matshow(observed[i])
    f.colorbar(im0, ax=axarr[0])
    axarr[0].set_title('observed, band = ' + str(bands[i]))


    im1 = axarr[1].matshow(recon_mean[0, i])
    f.colorbar(im1, ax=axarr[1])
    axarr[1].set_title('recon, band = ' + str(bands[i]))

    residual = recon_mean[0,i] - observed[i]
    foo = (residual / observed[i])
    im2 = axarr[2].matshow(foo, vmax = foo.abs().max(), vmin = - foo.abs().max(), cmap = plt.get_cmap('bwr'))
    f.colorbar(im2, ax=axarr[2])
    axarr[2].set_title('recon - obse, band = ' + str(bands[i]))

# Optimize

## first estimate the background

In [ ]:
background_bias = psf_transform_lib2.BackgroundBias(torch.zeros((2, 3)))
power_law_psf = psf_transform_lib2.PowerLawPSF(init_psf_params)

In [ ]:
background_optimizer = optim.LBFGS(list(background_bias.parameters()), 
                                   line_search_fn = 'strong_wolfe',
                                   max_iter = 20)

In [ ]:
def get_loss(background_bias, power_law_psf): 
    backgrounds = (sdss_hubble_data.sdss_background + background_bias.forward()).unsqueeze(0) 
    
    recon_mean, loss = get_psf_loss(sdss_hubble_data.sdss_image.unsqueeze(0), 
                        backgrounds, 
                        _locs, _fluxes, _n_stars, 
                        power_law_psf.forward())
    
    return recon_mean, loss

In [ ]:
get_loss(background_bias, power_law_psf)[1]

In [ ]:
def closure(): 
    background_optimizer.zero_grad()
    loss = get_loss(background_bias, power_law_psf)[1]    
    loss.backward()
    return loss

_ = background_optimizer.step(closure)

In [ ]:
recon_mean1, loss = get_loss(background_bias, power_law_psf)
print(loss)
recon_mean1 = recon_mean1.detach()

In [ ]:
plt.matshow(background_bias.forward().detach()[1])
plt.colorbar()

In [ ]:
for i in range(len(bands)): 
    f, axarr = plt.subplots(1, 3, figsize=(16, 4))

    observed = sdss_hubble_data.sdss_image
    im0 = axarr[0].matshow(observed[i])
    f.colorbar(im0, ax=axarr[0])
    axarr[0].set_title('observed, band = ' + str(bands[i]))


    im1 = axarr[1].matshow(recon_mean1[0, i])
    f.colorbar(im1, ax=axarr[1])
    axarr[1].set_title('recon, band = ' + str(bands[i]))

    residual = recon_mean1[0,i] - observed[i]
    foo = (residual / observed[i])
    im2 = axarr[2].matshow(foo, vmax = foo.abs().max(), vmin = - foo.abs().max(), cmap = plt.get_cmap('bwr'))
    f.colorbar(im2, ax=axarr[2])
    axarr[2].set_title('recon - obse, band = ' + str(bands[i]))

In [ ]:
psf_optimizer = optim.LBFGS(list(power_law_psf.parameters()), 
                            max_iter = 20, 
                           line_search_fn = 'strong_wolfe')

In [ ]:
list(background_bias.parameters())

In [ ]:
list(power_law_psf.parameters())

In [ ]:
def closure_psf(): 
    psf_optimizer.zero_grad()
    loss = get_loss(background_bias, power_law_psf)[1]    
    loss.backward()
    return loss

_ = psf_optimizer.step(closure_psf)


In [ ]:
recon_mean2, loss = get_loss(background_bias, power_law_psf)
print(loss)
recon_mean2 = recon_mean2.detach()

In [ ]:
for i in range(len(bands)): 
    f, axarr = plt.subplots(1, 3, figsize=(16, 4))

    observed = sdss_hubble_data.sdss_image
    im0 = axarr[0].matshow(observed[i])
    f.colorbar(im0, ax=axarr[0])
    axarr[0].set_title('observed, band = ' + str(bands[i]))


    im1 = axarr[1].matshow(recon_mean2[0, i])
    f.colorbar(im1, ax=axarr[1])
    axarr[1].set_title('recon, band = ' + str(bands[i]))

    residual = recon_mean2[0,i] - observed[i]
    foo = (residual / observed[i])
    im2 = axarr[2].matshow(foo, vmax = foo.abs().max(), vmin = - foo.abs().max(), cmap = plt.get_cmap('bwr'))
    f.colorbar(im2, ax=axarr[2])
    axarr[2].set_title('recon - obse, band = ' + str(bands[i]))

In [ ]:
plt.matshow(simulated_datasets_lib._trim_psf(power_law_psf.forward().detach(), 10)[0])
plt.colorbar()

In [ ]:
plt.matshow(simulated_datasets_lib._trim_psf(power_law_psf.forward().detach() - init_psf, 10)[0])
plt.colorbar()

In [ ]:
((recon_mean1 - observed)**2).mean()

In [ ]:
((recon_mean2 - observed)**2).mean()

# coordinate ascent in both

In [ ]:
old_loss = 1e16

In [ ]:
for i in range(4): 
    _ = background_optimizer.step(closure)
    _, loss = get_loss(background_bias, power_law_psf)
    print(loss)
    
    _ = psf_optimizer.step(closure_psf)
    
    _, loss = get_loss(background_bias, power_law_psf)
    print(loss)
    
    if (old_loss - loss) < 1e4: 
        break 
    
    old_loss = loss

In [ ]:
recon_final, loss = get_loss(background_bias, power_law_psf)
loss

In [ ]:
recon_final = recon_final.detach()

In [ ]:
list(background_bias.parameters())

In [ ]:
list(power_law_psf.parameters())

In [ ]:
for i in range(len(bands)): 
    f, axarr = plt.subplots(1, 3, figsize=(16, 4))

    observed = sdss_hubble_data.sdss_image
    im0 = axarr[0].matshow(observed[i])
    f.colorbar(im0, ax=axarr[0])
    axarr[0].set_title('observed, band = ' + str(bands[i]))


    im1 = axarr[1].matshow(recon_final[0, i])
    f.colorbar(im1, ax=axarr[1])
    axarr[1].set_title('recon, band = ' + str(bands[i]))

    residual = recon_final[0,i] - observed[i]
    foo = (residual / observed[i])
    im2 = axarr[2].matshow(foo, vmax = foo.abs().max(), vmin = - foo.abs().max(), cmap = plt.get_cmap('bwr'))
    f.colorbar(im2, ax=axarr[2])
    axarr[2].set_title('recon - obse, band = ' + str(bands[i]))

In [ ]:
plt.matshow(simulated_datasets_lib._trim_psf(power_law_psf.forward().detach(), 10)[0])
plt.colorbar()

In [ ]:
foo = simulated_datasets_lib._trim_psf(power_law_psf.forward().detach() - init_psf, 10)[0]
plt.matshow(foo)
plt.colorbar()

In [ ]:
power_law_psf.forward().min()

In [ ]:
init_psf.min()

In [ ]:
plt.matshow(recon_final[0, 0] - recon_mean1[0, 0])
plt.colorbar()

In [ ]:
np.save('../fits/results_2020-02-04/true_background_bias', 
        background_bias.forward().detach().numpy())

In [ ]:
np.save('../fits/results_2020-02-04/true_psf_params', 
        power_law_psf.params.data.numpy())

In [ ]:
list(power_law_psf.parameters())

In [ ]:
np.load('../fits/results_2020-02-04/true_background_bias.npy')

In [ ]:
np.load('../fits/results_2020-02-04/true_psf_params.npy')

In [ ]:
init_psf_params

In [ ]:
power_law_psf.params.data